# This file explores the ICRISAT district-level data and produces an interactive plot for it. No consideration is given to the hybrid methodology in this file. It is intended to be a standalone file that can be run independently of the rest of the project.

In [20]:
%matplotlib inline

In [21]:
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
# from matplotlib import rc

# rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# rc('text', usetex=True)

In [22]:
icrisat_path = '/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/other_databases/ICRISAT/'
shape_path = icrisat_path + '1966_districts_india/1966_districts_india.shp'
data_path = icrisat_path + 'ICRISAT-District Level Data_Apportioned.csv'

In [23]:
# Read the shapefile of district boundaries
districts = gpd.read_file(shape_path)

# Read the CSV containing crop data (which includes columns like 'district_name', 'year', 'crop', 'area', 'production', 'yield')
crop_data = pd.read_csv(data_path)

In [24]:
import os
os.getcwd()

'/Users/michaelfoley/Documents/Old_Documents/Climate/Crops/india_hybrid_methodology/mike'

# First we must match the names from the 1966 boundaries to the data

In [25]:
#Check names from both datasets
# For the first dataframe (CSV data)
selected_year = 2016
unique_csv_districts = pd.DataFrame(crop_data['Dist Name'].unique(), columns=['District'])
unique_csv_districts.to_csv('unique_csv_districts.csv', index=False)

# For the second dataframe (shapefile data)
unique_shape_districts = pd.DataFrame(districts['Dist_Name'].unique(), columns=['District'])
unique_shape_districts.to_csv('unique_shape_districts.csv', index=False)

# Print counts to verify
print(f"Number of unique districts in CSV: {len(unique_csv_districts)}")
print(f"Number of unique districts in shapefile: {len(unique_shape_districts)}")

Number of unique districts in CSV: 311
Number of unique districts in shapefile: 311


In [26]:
district_mapping = {
    # Basic spelling variations
    '24 Parganas': '24 Paraganas',
    'Ahmedabad': 'Ahmadabad',
    'Almorah': 'Almora',
    'Ananthapur': 'Anantapur',
    'Amarawati': 'Amravati',
    'Balasore': 'Baleshwar',
    'Banaskantha': 'Banas Kantha',
    'Barabanki': 'Bara Banki',
    'Beed': 'Bid',
    'Bhatinda': 'Bathinda',
    'Bilashpur': 'Bilaspur2',
    'Bilaspur': 'Bilaspur1',
    'Buldhana': 'Buldana',
    'Burdwan': 'Barddhaman',
    'Buland Shahar': 'Bulandshahr',
    'Chickmagalur': 'Chikmagalur',
    'Chittorgarh': 'Chittaurgarh',
    'Cooch Behar': 'Kochbihar',
    'Darjeeling': 'Darjiling',
    'Dehradun': 'Dehra Dun',
    'Eranakulam': 'Ernakulam',
    'Ferozpur': 'Firozpur',
    'Garhwal': 'Pauri Garhwal',
    'Hazaribagh': 'Hazaribag',
    'Hissar': 'Hisar',
    'Hooghly': 'Hugli',
    'Howrah': 'Haora',
    'Jalore': 'Jalor',
    'Jhunjhunu': 'Jhunjhunun',
    'Kanyakumari': 'Kanniyakumari',
    'Kheri': 'Lakhimpur Kheri',
    #'Koch Bihar': 'Kochbihar',
    'Kutch': 'Kachchh',
    'Lahul & Spiti': 'Lahul and Spiti',
    'Malda': 'Maldah',
    'Mayurbhanja': 'Mayurbhanj',
    'Mehsana': 'Mahesana',
    'Medinipur': 'Midnapore',
    'Midnapur': 'Midnapore',
    'Mirzpur': 'Mirzapur',
    'Mungair': 'Munger',
    'Nainital': 'Naini Tal',
    'Nasik': 'Nashik',
    'Palamau': 'Palamu',
    'Panchmahal': 'Panch Mahals',
    'Pithorgarh': 'Pithoragarh',
    'Purnea': 'Purnia',
    'Purulia': 'Puruliya',
    'Raigad': 'Raigarh2',
    'Raigarh': 'Raigarh1',
    'Ramananthapuram': 'Ramanathapuram',
    'Rae-Bareily': 'Rae Bareli',
    'Sabarkantha': 'Sabar Kantha',
    'Shimoge': 'Shimoga',
    'Swami Madhopur': 'Sawai Madhopur',
    'Dangs': 'The Dangs',
    #'Thirunelveli': 'Tirunelveli',
    'Tiruchirappalli': 'Tiruchchirappalli',
    'Thirunelveli': 'Tirunelveli Kattabo',
    #'Twenty Four Parganas': '24 Paraganas',
    'Uttar Kashi': 'Uttarkashi',
    'Yeotmal': 'Yavatmal',
    'Budaun': 'Badaun',
    
    # Districts with alternative names
    'Bijapur / Vijayapura': 'Bijapur',
    'Chengalpattu MGR / Kanchipuram': 'Chengalpattu',
    'Dakshina Kannada': 'Dakshin Kannad',
    'Uttara Kannada': 'Uttar Kannand',
    #'North Kanara': 'Uttar Kannand',
    #'South Kanara': 'Dakshin Kannad',
    'Gulbarga / Kalaburagi': 'Gulbarga',
    'Khandwa / East Nimar': 'East Nimar',
    'Khargone / West Nimar': 'Khargone',
    'Kodagu / Coorg': 'Kodagu',
    'Mahabubnagar': 'Mahbubnagar',
    'Mahendragarh / Narnaul': 'Mahendragarh',
    'North Cachar Hil / Dima hasao': 'North Cachar Hills',
    'Phulbani ( Kandhamal )': 'Phulbani',
    'Roopnagar / Ropar': 'Rupnagar',
    'S.P.S. Nellore': 'Nellore',
    'Santhal Paragana / Dumka': 'Santhal Paragana',
    'Seoni / Shivani': 'Seoni',
    'Shahabad (now part of Bhojpur district)': 'Shahabad',
    'Vadodara / Baroda': 'Vadodara',
    #'West Nimar': 'Khargone',
    #'Dumka': 'Santhal Paragana',
    'North Arcot / Vellore': 'North Arcot',
    'South Arcot / Cuddalore': 'South Arcot',
    #'Cuddalore': 'South Arcot',
    #'Vellore': 'North Arcot',
    'Kadapa YSR': 'Cuddapah',
    'Visakhapatnam': 'Vishakhapatnam',
    'The Nilgiris': 'Nilgiris',
    'Tiruchirapalli / Trichy': 'Tiruchchirappalli'
}

# # First filter out the NA district
# districts_no_na = districts[districts['Dist_Name'] != 'NA'].copy()

# # Apply the mapping to the crop data to match shapefile names
# crop_data['Dist Name'] = crop_data['Dist Name'].map(district_mapping).fillna(crop_data['Dist Name'])

In [27]:
# Reverse the district_mapping dictionary
reverse_mapping = {v: k for k, v in district_mapping.items()}

# First filter out the NA district
districts_no_na = districts[districts['Dist_Name'] != 'NA'].copy()

# Replace Uttaranchal with Uttarakhand
districts_no_na = districts_no_na.replace('Uttaranchal', 'Uttarakhand')

# Move to Saharanpur to Uttar Pradesh
districts_no_na.loc[districts_no_na['Dist_Name'] == 'Saharanpur', 'NAME_1'] = 'Uttar Pradesh'

In [28]:
# Apply the reverse mapping to the districts dataframe
districts_no_na['Dist_Name'] = districts_no_na['Dist_Name'].map(reverse_mapping).fillna(districts_no_na['Dist_Name'])
districts_no_na.to_file('/Users/michaelfoley/Library/CloudStorage/GoogleDrive-mfoley@g.harvard.edu/My Drive/Subnational_Yield_Database/data/processed/IND/icrisat_apportioned/icrisat_boundary_match.shp')

In [29]:
# Then merge
merged = districts_no_na.merge(crop_data, left_on="Dist_Name", right_on="Dist Name", how="left", indicator=True)

In [30]:
# Print districts that appear in shapefile but not in crop data
print("Districts in shapefile but not in crop data:")
left_only = merged[merged['_merge'] == 'left_only']['Dist_Name'].tolist()
print(left_only)

# Print districts that appear in crop data but don't match shapefile
print("\nDistricts in crop data that don't match shapefile:")
right_only = merged[merged['_merge'] == 'right_only']['Dist Name'].tolist()
print(right_only)

# Print count of districts in each dataset for comparison
print(f"\nNumber of districts in shapefile (excluding NA): {len(districts_no_na)}")
print(f"Number of successfully matched districts: {len(merged[merged['_merge'] == 'both'])}")

Districts in shapefile but not in crop data:
[]

Districts in crop data that don't match shapefile:
[]

Number of districts in shapefile (excluding NA): 310
Number of successfully matched districts: 16098


In [31]:
# Check for missing or zero wheat production values
year_crop_data = crop_data[crop_data['Year'] == selected_year].copy()

print("Districts with zero wheat production:")
print(year_crop_data[year_crop_data['WHEAT PRODUCTION (1000 tons)'] == 0]['Dist Name'].tolist())

print("\nDistricts with NaN wheat production:")
print(year_crop_data[year_crop_data['WHEAT PRODUCTION (1000 tons)'].isna()]['Dist Name'].tolist())

# Also check the value distribution
print("\nWheat production value summary:")
print(year_crop_data['WHEAT PRODUCTION (1000 tons)'].describe())

# Check if any districts in the merged data have missing wheat production
merged_year = merged[merged['Year'] == selected_year].copy()
print("\nNumber of districts in merged data with missing wheat production:")
print(merged_year['WHEAT PRODUCTION (1000 tons)'].isna().sum())

Districts with zero wheat production:
['Srikakulam', 'Visakhapatnam', 'East Godavari', 'West Godavari', 'Krishna', 'Guntur', 'S.P.S. Nellore', 'Ananthapur', 'Kadapa YSR', 'Chittoor', 'Warangal', 'Khammam', 'Karimnagar', 'Bangalore', 'Kolar', 'Tumkur', 'Mysore', 'Mandya', 'Hassan', 'Shimoge', 'Chickmagalur', 'Dakshina Kannada', 'Uttara Kannada', 'Kodagu / Coorg', 'Chengalpattu MGR / Kanchipuram', 'South Arcot / Cuddalore', 'North Arcot / Vellore', 'Salem', 'Coimbatore', 'Tiruchirapalli / Trichy', 'Thanjavur', 'Madurai', 'Ramananthapuram', 'Thirunelveli', 'The Nilgiris', 'Kanyakumari', 'Bombay', 'Raigad', 'Ratnagiri', 'North Cachar Hil / Dima hasao', 'Alappuzha', 'Kannur', 'Eranakulam', 'Kottayam', 'Kozhikode', 'Malappuram', 'Palakkad', 'Kollam', 'Thrissur', 'Thiruvananthapuram']

Districts with NaN wheat production:
[]

Wheat production value summary:
count     311.000000
mean      363.224502
std       545.721823
min         0.000000
25%         0.535000
50%       104.050000
75%       5

In [32]:
# Crops available in the dataset
available_crops = [name for name in merged.columns if 'YIELD' in name]

# Create an interactive plot

In [33]:
# Create lookup table
# List of crops (for dropdown menus, etc.)
crops = [
    "RICE", "WHEAT", "KHARIF SORGHUM", "RABI SORGHUM", "SORGHUM",
    "PEARL MILLET", "MAIZE", "FINGER MILLET", "BARLEY", "CHICKPEA",
    "PIGEONPEA", "MINOR PULSES", "GROUNDNUT", "SESAMUM", "RAPESEED AND MUSTARD",
    "SAFFLOWER", "CASTOR", "LINSEED", "SUNFLOWER", "SOYABEAN",
    "OILSEEDS", "SUGARCANE", "COTTON", "FRUITS", "VEGETABLES",
    "FRUITS AND VEGETABLES", "POTATOES", "ONION", "FODDER"
]

# Lookup table mapping each crop and indicator to its column name.
lookup_table = {
    "RICE": {
        "area": "RICE AREA (1000 ha)",
        "production": "RICE PRODUCTION (1000 tons)",
        "yield": "RICE YIELD (Kg per ha)"
    },
    "WHEAT": {
        "area": "WHEAT AREA (1000 ha)",
        "production": "WHEAT PRODUCTION (1000 tons)",
        "yield": "WHEAT YIELD (Kg per ha)"
    },
    "KHARIF SORGHUM": {
        "area": "KHARIF SORGHUM AREA (1000 ha)",
        "production": "KHARIF SORGHUM PRODUCTION (1000 tons)",
        "yield": "KHARIF SORGHUM YIELD (Kg per ha)"
    },
    "RABI SORGHUM": {
        "area": "RABI SORGHUM AREA (1000 ha)",
        "production": "RABI SORGHUM PRODUCTION (1000 tons)",
        "yield": "RABI SORGHUM YIELD (Kg per ha)"
    },
    "SORGHUM": {
        "area": "SORGHUM AREA (1000 ha)",
        "production": "SORGHUM PRODUCTION (1000 tons)",
        "yield": "SORGHUM YIELD (Kg per ha)"
    },
    "PEARL MILLET": {
        "area": "PEARL MILLET AREA (1000 ha)",
        "production": "PEARL MILLET PRODUCTION (1000 tons)",
        "yield": "PEARL MILLET YIELD (Kg per ha)"
    },
    "MAIZE": {
        "area": "MAIZE AREA (1000 ha)",
        "production": "MAIZE PRODUCTION (1000 tons)",
        "yield": "MAIZE YIELD (Kg per ha)"
    },
    "FINGER MILLET": {
        "area": "FINGER MILLET AREA (1000 ha)",
        "production": "FINGER MILLET PRODUCTION (1000 tons)",
        "yield": "FINGER MILLET YIELD (Kg per ha)"
    },
    "BARLEY": {
        "area": "BARLEY AREA (1000 ha)",
        "production": "BARLEY PRODUCTION (1000 tons)",
        "yield": "BARLEY YIELD (Kg per ha)"
    },
    "CHICKPEA": {
        "area": "CHICKPEA AREA (1000 ha)",
        "production": "CHICKPEA PRODUCTION (1000 tons)",
        "yield": "CHICKPEA YIELD (Kg per ha)"
    },
    "PIGEONPEA": {
        "area": "PIGEONPEA AREA (1000 ha)",
        "production": "PIGEONPEA PRODUCTION (1000 tons)",
        "yield": "PIGEONPEA YIELD (Kg per ha)"
    },
    "MINOR PULSES": {
        "area": "MINOR PULSES AREA (1000 ha)",
        "production": "MINOR PULSES PRODUCTION (1000 tons)",
        "yield": "MINOR PULSES YIELD (Kg per ha)"
    },
    "GROUNDNUT": {
        "area": "GROUNDNUT AREA (1000 ha)",
        "production": "GROUNDNUT PRODUCTION (1000 tons)",
        "yield": "GROUNDNUT YIELD (Kg per ha)"
    },
    "SESAMUM": {
        "area": "SESAMUM AREA (1000 ha)",
        "production": "SESAMUM PRODUCTION (1000 tons)",
        "yield": "SESAMUM YIELD (Kg per ha)"
    },
    "RAPESEED AND MUSTARD": {
        "area": "RAPESEED AND MUSTARD AREA (1000 ha)",
        "production": "RAPESEED AND MUSTARD PRODUCTION (1000 tons)",
        "yield": "RAPESEED AND MUSTARD YIELD (Kg per ha)"
    },
    "SAFFLOWER": {
        "area": "SAFFLOWER AREA (1000 ha)",
        "production": "SAFFLOWER PRODUCTION (1000 tons)",
        "yield": "SAFFLOWER YIELD (Kg per ha)"
    },
    "CASTOR": {
        "area": "CASTOR AREA (1000 ha)",
        "production": "CASTOR PRODUCTION (1000 tons)",
        "yield": "CASTOR YIELD (Kg per ha)"
    },
    "LINSEED": {
        "area": "LINSEED AREA (1000 ha)",
        "production": "LINSEED PRODUCTION (1000 tons)",
        "yield": "LINSEED YIELD (Kg per ha)"
    },
    "SUNFLOWER": {
        "area": "SUNFLOWER AREA (1000 ha)",
        "production": "SUNFLOWER PRODUCTION (1000 tons)",
        "yield": "SUNFLOWER YIELD (Kg per ha)"
    },
    "SOYABEAN": {
        "area": "SOYABEAN AREA (1000 ha)",
        "production": "SOYABEAN PRODUCTION (1000 tons)",
        "yield": "SOYABEAN YIELD (Kg per ha)"
    },
    "OILSEEDS": {
        "area": "OILSEEDS AREA (1000 ha)",
        "production": "OILSEEDS PRODUCTION (1000 tons)",
        "yield": "OILSEEDS YIELD (Kg per ha)"
    },
    "SUGARCANE": {
        "area": "SUGARCANE AREA (1000 ha)",
        "production": "SUGARCANE PRODUCTION (1000 tons)",
        "yield": "SUGARCANE YIELD (Kg per ha)"
    },
    "COTTON": {
        "area": "COTTON AREA (1000 ha)",
        "production": "COTTON PRODUCTION (1000 tons)",
        "yield": "COTTON YIELD (Kg per ha)"
    },
    # The following crops only have an area measure.
    "FRUITS": {
        "area": "FRUITS AREA (1000 ha)"
    },
    "VEGETABLES": {
        "area": "VEGETABLES AREA (1000 ha)"
    },
    "FRUITS AND VEGETABLES": {
        "area": "FRUITS AND VEGETABLES AREA (1000 ha)"
    },
    "POTATOES": {
        "area": "POTATOES AREA (1000 ha)"
    },
    "ONION": {
        "area": "ONION AREA (1000 ha)"
    },
    "FODDER": {
        "area": "FODDER AREA (1000 ha)"
    }
}

In [34]:
# import json
# from shapely.geometry import mapping
# import plotly.express as px

# # Filter for a particular year and crop as an example:
# selected_year = 2010
# selected_crop = "WHEAT"  # for example
# selected_indicator = "production"  # could be 'area', 'production', or 'yield'

# column_keyword = lookup_table[selected_crop][selected_indicator]
# # This would yield: "WHEAT PRODUCTION (1000 tons)"

# # Filter the merged DataFrame:
# filtered = merged[merged["Year"] == selected_year]

# # First, let's identify districts with missing data
# districts_with_nan = filtered[filtered[column_keyword].isna()]["Dist_Name"]
# print("Districts with missing data:")
# print(districts_with_nan.tolist())

# # For the visualization, create a new column that includes a special value for NaN
# filtered['production_with_nan'] = filtered[column_keyword].fillna(-999)

# # Add indicator column
# filtered['Indicator'] = selected_indicator  # Every row will have the same value
# filtered = filtered.to_crs("EPSG:4326")

# # Use GeoDataFrame's built-in to_json() method with the correct parameters
# # Convert to GeoJSON format using __geo_interface__
# geojson_data = filtered.__geo_interface__


# # Create the map
# fig = px.choropleth(
#     filtered,
#     geojson=geojson_data,
#     locations="Dist_Name",  # this should match the key in your GeoJSON features
#     featureidkey="properties.Dist_Name",
#     color='production_with_nan',
#     #center={"lat": 20.5937, "lon": 78.9629},  # roughly center of India
#     #scope="asia",
#     # color_continuous_scale=[
#     #     [0, "red"],          # For our NaN values (-999)
#     #     [0.1, "purple"],     # Start of actual data
#     #     [1, "yellow"]        # Max values
#     # ],
#     range_color=[0, 131],
#     hover_data={
#         "Dist_Name": True,
#         "Indicator": True,
#     }
# )

# fig.update_layout(
#     width=1200,
#     height=800,
#     geo=dict(
#         visible=True,
#         projection_type='equirectangular',
#         lonaxis_range=[68, 98],
#         lataxis_range=[6, 38],
#         showframe=False,  # Remove frame
#         showcoastlines=False,  # Remove coastlines
#         #showcountries=True,  # Show country boundaries
#         countrycolor='Black',  # Set country boundary color
#         countrywidth=1,  # Set country boundary width
#         bgcolor='rgba(0,0,0,0)',  # Transparent background
#         landcolor='rgba(0,0,0,0)',  # Transparent land
#         oceancolor='rgba(0,0,0,0)'  # Transparent ocean
#     )
# )

# fig.show()

In [35]:
import os
import dash
from dash import Dash, dcc, html, Input, Output, ctx
import plotly.express as px
import pandas as pd

# -----------------------------------------------------------
# NOTE: Ensure that you have loaded your data (crop_data, merged, lookup_table)
# For example:
# crop_data = pd.read_csv("crop_data.csv")
# merged = pd.read_csv("merged_data.csv")
# lookup_table = ...  # however you load/build this dictionary
# -----------------------------------------------------------

# Initialize the Dash app
app = Dash(__name__)
server = app.server  # Expose the Flask server (required for Gunicorn)

# Get unique years for the slider (example assumes crop_data is loaded)
years = sorted(crop_data['Year'].unique())

# Define crops (as provided)
crops = [
    "RICE", "WHEAT", "KHARIF SORGHUM", "RABI SORGHUM", "SORGHUM",
    "PEARL MILLET", "MAIZE", "FINGER MILLET", "BARLEY", "CHICKPEA",
    "PIGEONPEA", "MINOR PULSES", "GROUNDNUT", "SESAMUM", "RAPESEED AND MUSTARD",
    "SAFFLOWER", "CASTOR", "LINSEED", "SUNFLOWER", "SOYABEAN",
    "OILSEEDS", "SUGARCANE", "COTTON"
]

# Precompute filtered data for each year
precomputed_data = {}
for year in years:
    year_data = merged[merged['Year'] == year].copy()
    precomputed_data[year] = year_data

# Precompute value ranges for each crop and indicator
value_ranges = {}
for crop in crops:
    value_ranges[crop] = {}
    for indicator in ['production', 'area', 'yield']:
        column = lookup_table[crop][indicator]
        if column in merged.columns:
            min_val = merged[column].min()
            max_val = merged[column].max()
            value_ranges[crop][indicator] = (min_val, max_val)

# Create the layout
app.layout = html.Div([
    html.H1("India Agricultural Data Visualization"),
    
    # Dropdown for crop selection
    html.Div([
        html.Label('Select Crop'),
        dcc.Dropdown(
            id='crop-dropdown',
            options=[{'label': crop, 'value': crop} for crop in crops],
            value='WHEAT'
        ),
    ]),
    
    # Dropdown for indicator selection
    html.Div([
        html.Label('Select Indicator'),
        dcc.Dropdown(
            id='indicator-dropdown',
            options=[
                {'label': 'Production (1000 tons)', 'value': 'production'},
                {'label': 'Area (1000 ha)', 'value': 'area'},
                {'label': 'Yield (Kg per ha)', 'value': 'yield'}
            ],
            value='production'
        ),
    ]),
    
    # Slider and play button controls
    html.Div([
        html.Button('Play', id='play-button', n_clicks=0),
        html.Button('Pause', id='pause-button', n_clicks=0),
        dcc.Slider(
            id='year-slider',
            min=min(years),
            max=max(years),
            value=2010,
            marks={str(year): str(year) for year in years[::5]},
            step=1
        ),
        dcc.Interval(
            id='interval-component',
            interval=2000,  # 2 seconds
            n_intervals=0,
            disabled=True
        ),
    ]),
    
    # Graph
    dcc.Graph(id='choropleth-map')
])

# Callback to control animation
@app.callback(
    Output('interval-component', 'disabled'),
    [Input('play-button', 'n_clicks'),
     Input('pause-button', 'n_clicks')],
    prevent_initial_call=True
)
def toggle_animation(play_clicks, pause_clicks):
    if not ctx.triggered:
        return True
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    return False if button_id == 'play-button' else True

# Callback to update the slider on interval
@app.callback(
    Output('year-slider', 'value'),
    [Input('interval-component', 'n_intervals'),
     Input('year-slider', 'value')],
    prevent_initial_call=True
)
def update_year_on_interval(n_intervals, current_year):
    return min(years) if current_year >= max(years) else current_year + 1

# Update the map callback
@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('crop-dropdown', 'value'),
     Input('indicator-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_map(selected_crop, selected_indicator, selected_year):
    column_keyword = lookup_table[selected_crop][selected_indicator]
    filtered = precomputed_data[selected_year]
    value_range = value_ranges[selected_crop][selected_indicator]
    
    fig = px.choropleth(
        filtered,
        geojson=filtered.__geo_interface__,
        locations=filtered.index,
        featureidkey="id",
        color=column_keyword,
        title=f"{selected_crop} {selected_indicator} in India Districts ({selected_year})",
        color_continuous_scale="Viridis",
        range_color=value_range,
        hover_data={
            'Dist_Name': True,
            column_keyword: ':.2f'
        },
        labels={
            column_keyword: selected_indicator.capitalize(),
            'Dist_Name': 'District'
        }
    )
    
    fig.update_layout(
        width=800,
        height=600,
        geo=dict(
            visible=True,
            projection_type='equirectangular',
            lonaxis_range=[68, 98],
            lataxis_range=[6, 38],
            showframe=False,
            showcoastlines=False,
            countrycolor='Black',
            countrywidth=1,
            bgcolor='rgba(0,0,0,0)',
            landcolor='rgba(0,0,0,0)',
            oceancolor='rgba(0,0,0,0)'
        ),
        transition_duration=150
    )
    return fig

# Run the app (for local development)
if __name__ == '__main__':
    port = int(os.environ.get("PORT", 8080))
    app.run(debug=False, host='0.0.0.0', port=port)